In [1]:
import EOSutils as meos

import numpy as np
import matplotlib.pyplot as plt

import matplotlib.style
import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.cm as cmx
from mpl_toolkits.axes_grid1 import make_axes_locatable

mpl.style.use('classic')

from astropy.table import Table
from astropy import units as u
from astropy.constants import G

import mesa_helper as mh
import os
import shutil

%matplotlib inline

In [2]:
print(2.2*u.kg.to(u.jupiterMass))

1.159038770735926e-27


In [3]:
def write_MESA_table_datacube(datacube, outfilename, version=1, X=1.0, Z=0.0):
    """
    Convert a table of shape (nQ, nT, 19) into MESA's bizarre (T, Q) format and save to file.
    datacube[:,:,-2] is log10Q
    datacube[:,:,-1] is log10rho
    """
    numlogTs = 121
    numlogQs = 349

    Tarr = datacube[0,:,0]
    Qarr = datacube[:,0,-2]
    
    logTmin = np.min(Tarr)
    logTmax = np.max(Tarr)
    dellogT = Tarr[1] - Tarr[0]

    # only use Qs <= 5.4, because this is the value that corresponds to the boundary between physical and unphysical values in the CMS pure H table
    logQmin = np.min(Qarr)
    logQmax = np.max(Qarr)
    dellogQ = Qarr[1] - Qarr[0]
    
    header0keys = ['version','X','Z','num logTs','logT min','logT max','del logT','num logQs','logQ min','logQ max','del logQ']
    header0vals = [version,X,Z,numlogTs,logTmin,logTmax,dellogT,numlogQs,logQmin,logQmax,dellogQ]

    header0=""
    for k in header0keys:
        kstr = '{:>14}'.format(k)
        header0 = header0 + kstr

    header1=""
    for v in header0vals:
        vstr = '{:>14}'.format(str(np.round(v,6)))
        header1 = header1 + vstr

    header2keys =['logT','logPgas','logE','logS','chiRho','chiT','Cp','Cv','dE_dRho','dS_dT','dS_dRho','mu ','log_free_e','gamma1','gamma3','grad_ad','eta']

    with open(outfilename, 'w') as outfile:
        outfile.write(header0)
        outfile.write('\n')
        outfile.write(header1)
        outfile.write('\n')

        for i, logQ in enumerate(Qarr):
            outfile.write('\n')
            outfile.write('       logQ = logRho - 2*logT + 12')
            outfile.write('\n')
            logQstr = f"{np.round(logQ,6):#.4g}"
            logQstr = logQstr.split(".")[0] + '.' + logQstr.split(".")[1][0:2]
            outfile.write('{:>12}'.format(logQstr))
            outfile.write('\n')
            outfile.write('\n')
            header2=''
            for l, key in enumerate(header2keys):
                if l==0:
                    kstr = key
                elif l > 0 and l <= 3:
                    kstr = '{:>10}'.format(key)
                elif l > 3 and l <= 10:
                    kstr = '{:>14}'.format(key)
                else:
                    kstr = '{:>10}'.format(key)    
                header2 = header2 + kstr
            outfile.write(header2)
            outfile.write('\n')

            # write data here

            # get columns
            # attributes are named:
            # log10Qgrid, log10Tgrid, log10Pgrid, log10Ugrid, log10Sgrid, chiRho, chiT, Cp, Cv, dE_drho_T, dS_dT_rho, dS_drho_T, mu, log_free_e, gamma1, gamma3, grad_ad, eta
            # every one of them is a (281, 121) array (281 rho values, 121 T values)

            for j in range(numlogTs):
                rowList = list(datacube[i,j,:17])
                           
                rowStr=""
                # from here: want different formatting for the different columns, actually 
                #logT   logPgas      logE      logS       chiRho         chiT           Cp           Cv      dE_dRho        dS_dT      dS_dRho      mu  log_free_e   gamma1    gamma3   grad_ad       eta
                #2.10  -4.28120   9.90528   9.17145  9.88041E-01  1.03588E+00  1.14092E+08  6.83968E+07 -1.90043E+22  5.43295E+05 -4.35851E+21  0.50000 -99.00000   1.64814   1.63724   0.38664  -20.00000

                for k, r in enumerate(rowList):
                    if k == 0:
                        rstr = "{:.2f}".format(r)
                    elif k > 0 and k <= 3:
                        rstr = ' ' + "{:.5f}".format(r)
                        if 'nan' in rstr:
                            #print(rstr)
                            rnew = np.median(datacube[:,:,k][np.isfinite(datacube[:,:,k])])
                            rstr = rstr = ' ' + "{:.5f}".format(rnew)
                            #print(rstr)
                        rstr = '{:>10}'.format(rstr)
                    elif k > 3 and k <= 10:
                        try:
                            rstr = ' ' + meos.MESA_format_e(r)
                        except IndexError:
                            print(k,header2keys[k],r,np.sign(r))
                            rfinite = np.median(datacube[:,:,k][np.isfinite(datacube[:,:,k])])
                            print(rfinite)
                            rstr = ' ' + meos.MESA_format_e(rfinite)
                            
                        rstr = '{:>14}'.format(rstr)
                    else:
                        rstr = ' ' + "{:.5f}".format(r)
                        if 'nan' in rstr:
                            #print(rstr)
                            rnew = np.median(datacube[:,:,k][np.isfinite(datacube[:,:,k])])
                            rstr = rstr = ' ' + "{:.5f}".format(rnew)
                            #print(rstr)
                        rstr = '{:>10}'.format(rstr)
                    
                    rowStr = rowStr + rstr
                    
                outfile.write(rowStr)
                outfile.write('\n')

            
            outfile.write('\n')
            #outfile.write('\n')

    
    return

In [4]:
z00x100 = meos.read_MESAtable("./meos_MESAformat_tables/mesa-planetblend_00z100x.data")
z00x100old = meos.read_MESAtable("./my_MESAformat_tables/mesa-planetblend_00z100x.data")


print(np.shape(z00x100))
print(np.shape(z00x100old))

print(np.shape(z00x100[:,:,16]))

print(z00x100[:,:,16])
print(z00x100old[:,:,16])

z00x100[:,:,16] = z00x100old[:,:,16]


write_MESA_table_datacube(z00x100, outfilename="./test.data", version=1, X=1.0, Z=0.0)

(349, 121, 19)
(349, 121, 19)
(349, 121)
[[-34.3305  -34.27086 -34.21157 ... -23.37163 -23.37157 -23.3715 ]
 [-34.21531 -34.15569 -34.0964  ... -23.37163 -23.37157 -23.3715 ]
 [-34.10013 -34.04051 -33.98124 ... -23.37163 -23.37157 -23.3715 ]
 ...
 [ 47.8096   49.68135  51.62627 ... 196.47127 179.731   164.31462]
 [ 51.62627  53.64723  55.74721 ... 199.05491 182.03408 166.36772]
 [ 55.74721  57.9293   60.19672 ... 201.63854 184.33716 168.42081]]
[[-34.3305  -34.27086 -34.21157 ... -23.37163 -23.37157 -23.3715 ]
 [-34.21531 -34.15569 -34.0964  ... -23.37163 -23.37157 -23.3715 ]
 [-34.10013 -34.04051 -33.98124 ... -23.37163 -23.37157 -23.3715 ]
 ...
 [ 47.8096   49.68135  51.62627 ... 196.47127 179.731   164.31462]
 [ 51.62627  53.64723  55.74721 ... 199.05491 182.03408 166.36772]
 [ 55.74721  57.9293   60.19672 ... 201.63854 184.33716 168.42081]]


In [5]:
newEOSdir = "./meos_MESAformat_tables"
newEOSfiles = [f for f in os.listdir(newEOSdir) if os.path.isfile(os.path.join(newEOSdir, f))]

oldEOSdir = "./my_MESAformat_tables"
oldEOSfiles = [f for f in os.listdir(oldEOSdir) if os.path.isfile(os.path.join(oldEOSdir, f)) and 'planetblend' in f]

for nfile in newEOSfiles:
    ncomp = nfile.split("_")[-1]

    if "protosolar" in nfile:
        # there is no protosolar composition file in my_MESAformat_tables; have to treat this one separately
        continue
    else:
        xi = float(ncomp.split("z")[1].split("x")[0])/100.
        zi = float(ncomp.split("z")[0])/100.

    
    for ofile in oldEOSfiles:
        ocomp = ofile.split("_")[-1]

        if ocomp == ncomp:

            #print(os.path.join(newEOSdir,nfile))
            #print(os.path.join(oldEOSdir,ofile))
            
            table_with_correct_cols0to15 = meos.read_MESAtable(os.path.join(newEOSdir,nfile))
            table_with_correct_eta = meos.read_MESAtable(os.path.join(oldEOSdir,ofile))

            table_with_correct_cols0to15[:,:,16] = table_with_correct_eta[:,:,16]
            print(nfile)
            write_MESA_table_datacube(table_with_correct_cols0to15, outfilename=os.path.join(newEOSdir,nfile), version=1, X=xi, Z=zi)
     

mesa-planetblend-control_s=5_sf=0.1_10z50x.data
mesa-planetblend-control_s=1_sf=0.25_10z40x.data
mesa-planetblend-control_s=2_sf=0.05_20z60x.data
mesa-planetblend-control_s=5_sf=0.05_20z80x.data
mesa-planetblend-TC_s=5_00z50x.data
mesa-planetblend-control_s=2_sf=0.1_10z80x.data
mesa-planetblend-control_s=2_sf=0.25_00z20x.data
mesa-planetblend-control_s=1_sf=0.05_100z00x.data
mesa-planetblend-TC_s=1_20z50x.data
mesa-planetblend-control_s=5_sf=0.25_00z00x.data
mesa-planetblend-control_s=1_sf=0.1_20z10x.data
mesa-planetblend-control_s=1_sf=0.25_10z80x.data
mesa-planetblend-TC_s=2_00z00x.data
mesa-planetblend-control_s=5_sf=0.1_10z90x.data
mesa-planetblend-control_s=2_sf=0.1_10z40x.data
mesa-planetblend-TC_s=5_00z90x.data
mesa-planetblend-control_s=5_sf=0.05_20z40x.data
mesa-planetblend-TC_s=1_10z20x.data
mesa-planetblend-control_s=2_sf=0.1_20z30x.data
mesa-planetblend-control_s=5_sf=0.05_10z30x.data
mesa-planetblend-control_s=1_sf=0.1_10z60x.data
mesa-planetblend-control_s=2_sf=0.05_10z10

In [6]:
# protosolar composition only
newEOSdir = "./meos_MESAformat_tables"
protosolar_files = [f for f in os.listdir(newEOSdir) if os.path.isfile(os.path.join(newEOSdir, f)) and 'protosolar' in f]

protosolar_file_with_eta = "./meos_MESAformat_tables/mesa-planetblend_protosolar.data"

for nfile in newEOSfiles:

    fname = os.path.join(newEOSdir, nfile)

    if "protosolar" in fname:
        print(fname)
        if fname == protosolar_file_with_eta:
            print("yes")
            continue
        else:
            table_with_correct_cols0to15 = meos.read_MESAtable(fname)
            table_with_correct_eta = meos.read_MESAtable(protosolar_file_with_eta)

            table_with_correct_cols0to15[:,:,16] = table_with_correct_eta[:,:,16]
            print(nfile)
            write_MESA_table_datacube(table_with_correct_cols0to15, outfilename=fname, version=1, X=0.706, Z=0.017)
     
   

./test_cleanup_mesaEOSnotebook/mesa-planetblend-TC_s=5_protosolar.data
mesa-planetblend-TC_s=5_protosolar.data
./test_cleanup_mesaEOSnotebook/mesa-planetblend-control_s=2_sf=0.05_protosolar.data
mesa-planetblend-control_s=2_sf=0.05_protosolar.data
./test_cleanup_mesaEOSnotebook/mesa-planetblend-TC_s=2_protosolar.data
mesa-planetblend-TC_s=2_protosolar.data
./test_cleanup_mesaEOSnotebook/mesa-planetblend-control_s=1_sf=0.05_protosolar.data
mesa-planetblend-control_s=1_sf=0.05_protosolar.data
./test_cleanup_mesaEOSnotebook/mesa-planetblend-control_s=5_sf=0.05_protosolar.data
mesa-planetblend-control_s=5_sf=0.05_protosolar.data
./test_cleanup_mesaEOSnotebook/mesa-planetblend-TC_s=1_protosolar.data
mesa-planetblend-TC_s=1_protosolar.data
./test_cleanup_mesaEOSnotebook/mesa-planetblend-control_s=2_sf=0.25_protosolar.data
mesa-planetblend-control_s=2_sf=0.25_protosolar.data
./test_cleanup_mesaEOSnotebook/mesa-planetblend-control_s=1_sf=0.25_protosolar.data
mesa-planetblend-control_s=1_sf=0.2

In [10]:
'''
Xarr = np.arange(0.0,1.1,0.1)
Zarr = np.arange(0.0,1.1,0.1)

for i, xi in enumerate(Xarr):
    for k, zi in enumerate(Zarr):

        if (xi + zi) > 1.0:
            continue
        else:
            print(np.round(zi,1),np.round(xi,1))
            print(k,i)
            
            savefilename = './my_MESAformat_tables/mesa-planetblend_{0}0z{1}0x.data'.format(k,i)
            
            table_with_correct_cols0to15 = meos.read_MESAtable(savefilename)
            table_with_correct_eta = read_MESAtable("./my_MESAformat_tables_somenegativevalues/mesa-planetblend_{0}0z{1}0x.data".format(k,i))

            table_with_correct_cols0to15[:,:,16] = table_with_correct_eta[:,:,16]

            write_MESA_table_datacube(table_with_correct_cols0to15, outfilename=savefilename, version=1, X=xi, Z=zi)
'''         

0.0 0.0
0 0
0.1 0.0
1 0
0.2 0.0
2 0
0.3 0.0
3 0
0.4 0.0
4 0
0.5 0.0
5 0
0.6 0.0
6 0
0.7 0.0
7 0
0.8 0.0
8 0
0.9 0.0
9 0
1.0 0.0
10 0
0.0 0.1
0 1
0.1 0.1
1 1
0.2 0.1
2 1
0.3 0.1
3 1
0.4 0.1
4 1
0.5 0.1
5 1
0.6 0.1
6 1
0.7 0.1
7 1
0.8 0.1
8 1
0.9 0.1
9 1
0.0 0.2
0 2
0.1 0.2
1 2
0.2 0.2
2 2
0.3 0.2
3 2
0.4 0.2
4 2
0.5 0.2
5 2
0.6 0.2
6 2
0.7 0.2
7 2
0.8 0.2
8 2
0.0 0.3
0 3
0.1 0.3
1 3
0.2 0.3
2 3
0.3 0.3
3 3
0.4 0.3
4 3
0.5 0.3
5 3
0.6 0.3
6 3
0.7 0.3
7 3
0.0 0.4
0 4
0.1 0.4
1 4
0.2 0.4
2 4
0.3 0.4
3 4
0.4 0.4
4 4
0.5 0.4
5 4
0.6 0.4
6 4
0.0 0.5
0 5
0.1 0.5
1 5
0.2 0.5
2 5
0.3 0.5
3 5
0.4 0.5
4 5
0.5 0.5
5 5
0.0 0.6
0 6
0.1 0.6
1 6
0.2 0.6
2 6
0.3 0.6
3 6
0.4 0.6
4 6
0.0 0.7
0 7
0.1 0.7
1 7
0.2 0.7
2 7
0.3 0.7
3 7
0.0 0.8
0 8
0.1 0.8
1 8
0.2 0.8
2 8
0.0 0.9
0 9
0.1 0.9
1 9
0.0 1.0
0 10


In [11]:
table_with_correct_cols0to15 = read_MESAtable('./my_MESAformat_tables/mesa-iron_100z00x.data')
table_with_correct_eta = read_MESAtable("./my_MESAformat_tables_somenegativevalues/mesa-iron_100z00x.data")
table_with_correct_cols0to15[:,:,16] = table_with_correct_eta[:,:,16]
write_MESA_table_datacube(table_with_correct_cols0to15, outfilename='./my_MESAformat_tables/mesa-iron_100z00x.data', version=1, X=0, Z=1)

table_with_correct_cols0to15 = read_MESAtable('./my_MESAformat_tables/mesa-rock_100z00x.data')
table_with_correct_eta = read_MESAtable("./my_MESAformat_tables_somenegativevalues/mesa-rock_100z00x.data")
table_with_correct_cols0to15[:,:,16] = table_with_correct_eta[:,:,16]
write_MESA_table_datacube(table_with_correct_cols0to15, outfilename='./my_MESAformat_tables/mesa-rock_100z00x.data', version=1, X=0, Z=1)

table_with_correct_cols0to15 = read_MESAtable('./my_MESAformat_tables/mesa-h2o_100z00x.data')
table_with_correct_eta = read_MESAtable("./my_MESAformat_tables_somenegativevalues/mesa-h2o_100z00x.data")
table_with_correct_cols0to15[:,:,16] = table_with_correct_eta[:,:,16]
write_MESA_table_datacube(table_with_correct_cols0to15, outfilename='./my_MESAformat_tables/mesa-h2o_100z00x.data', version=1, X=0, Z=1)
            